В main.py код для связи Томиты и Питона. Чтобы воспроизвести эксперимент, нужно скачать бинарник Томиты с https://tech.yandex.ru/tomita/ и положить его в папку tomita.

In [23]:
%run -n main.py

# Сравнение скорости Наташи и Томиты при извлечении имён

Попробуем сравнить производительность Наташи и Томиты на задаче извлечения имён. Грамматики для Томиты для работы с именами лежат в tomita/algfio. Они повторяют пример https://github.com/yandex/tomita-parser/tree/master/examples/algfio из репозитория Томиты. Наташа использует стандартный NamesExtractor. Будем делать замеры на двух датасетах: отзывы о воспистателях детских садов и тексты новостей c lenta.ru.

Томита извлекает имена в ~30 раз быстрее, чем Наташа. Если запускать Наташу под Pypy, разрыв сокращается: Томита в ~6 раз быстрее.

<table>
    <th>
        <td></td>
        <td>Наташа, текстов в секунду</td>
        <td>Томита</td>
        <td>Наташа под Pypy</td>
        <td>Томита / Наташа</td>
        <td>Томита / Наташа под Pypy</td>
    </th>
    <tr>
        <td>Отзывы<td>
        <td>7.4 </td>
        <td>212.8</td>
        <td>31.2</td>
        <td>28.5</td>
        <td>6.8</td>
    <tr>
    <tr>
        <td>Новости<td>
        <td>4.6</td>
        <td>137</td>
        <td>23.3</td>
        <td>29.8</td>
        <td>5.9</td>
    <tr>
</table>

На текстах с отзывами у обоих решений очень хорошее качество — точность ~95%. В новостях чаще встречаются иностранные имена, Наташа с ними работает плохо, качество получается ~75%. Визуально Томита справляется лучше, точная оценка не делалась.

С задачей извлечения имён Томита справляется лучше и по производительности и по качеству разметки.

# Отзывы

In [18]:
reviews = list(load_lines(REVIEWS))

Производительность Наташи 7.4 текстов в секунду (1000 текстов за 135 секунд)

In [21]:
%%time
extractor = NamesExtractor()
records1 = [extractor(_) for _ in reviews]

CPU times: user 2min 12s, sys: 1.84 s, total: 2min 13s
Wall time: 2min 15s


Производительность Томиты 212.8 текстов в секунду (10 000 текстов за 47 секунд)

In [27]:
%%time
records2 = list(run_algfio_tomita(reviews * 10))

CPU times: user 2.65 s, sys: 2.08 s, total: 4.73 s
Wall time: 47.1 s


Попробуем ещё запустить Наташу под Pypy. 

```bash
time pypy pypy.py reviews.txt
# 26,90s user 1,17s system 87% cpu 31,991 total
```

Получается 31.2 текста в секунду

Убедимся, что оба решения извлекают что-то разумное. В каждой паре первой идёт разметка Наташи, второй — Томиты. Видно, что в большинстве случаев ответы совпадают. Бывает ошибки и с той и с другой стороны.

Ошибки Томиты:
- Не проверяется согласование частей имени: Светлана Николаевне, Светлане Юрьевне Федоровне
- Слова, которые не являются именами: "... с весенним праздником 8 МАРТА", "... проявили ЛЮБОВЬ". "Марта" и "любовь" здесь не имена. В "... группы Колокольчики Галине Ивановне" "Колокольчики" не фамилия.

Ошибки Наташи:
- Не ловит сложные фамилии: "Мацаль Елене", "Лагуненок Валентине", "Войтик Марине", "А. Дистервег"
- Не очень сложные фамилии тоже иногда не ловит: "Прусаковой Татьяны Николаевны", "Цаплина Елена", "Евгений Филькин"

In [35]:
%run -n main.py
records = list(zip(reviews, records1, records2))
random_seed(41)
for text, matches1_, matches2_ in sample(records, 10):
    spans1 = sorted(_.span for _ in matches1_)
    spans2 = sorted(_.span for _ in matches2_)
    if spans1 != spans2:
        show_markup(text, spans1)
        print('-------------')
        show_markup(text, spans2)
        print('=============')

-------------


-------------


-------------


-------------


-------------


# Новости

In [31]:
news = list(load_lines(NEWS))

4.6 текст/c

In [42]:
%%time
extractor = NamesExtractor()
records1 = [extractor(_) for _ in log_progress(news)]

CPU times: user 3min 33s, sys: 1.83 s, total: 3min 35s
Wall time: 3min 36s


137 текст/c

In [45]:
%%time
records2 = list(run_algfio_tomita(news * 10))

CPU times: user 3.43 s, sys: 2.03 s, total: 5.45 s
Wall time: 1min 13s


```bash
time pypy pypy.py news.txt
# 34,99s user 1,39s system 84% cpu 42,884 total
```

23.3 текст/c

In [49]:
%run -n main.py
records = list(zip(news, records1, records2))
random_seed(41)
for text, matches1_, matches2_ in sample(records, 10):
    spans1 = sorted(_.span for _ in matches1_)
    spans2 = sorted(_.span for _ in matches2_)
    if spans1 != spans2:
        show_markup(text, spans1)
        print('-------------')
        show_markup(text, spans2)
        print('=============')

-------------


-------------


-------------


-------------


-------------


-------------


-------------
